In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

In [2]:
xy_df = pd.read_csv('iris.csv', delimiter=',')
x_data = xy_df.iloc[:,0:-1].to_numpy()
x_data.shape

(150, 4)

In [4]:
xy_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
 pd.get_dummies(xy_df.iloc[:,-1]).values.argmax(1).astype(np.float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
      dtype=float32)

In [55]:
# Creating Network

class Model(torch.nn.Module):
    
    def __init__(self):
        """
        Initialize the network with the layers
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(4, 10)
        self.l2 = torch.nn.Linear(10, 8)
        self.l3 = torch.nn.Linear(8, 3)
        
        self.softmax = torch.nn.Softmax()
        
    def forward(self, x):
        """
        Defining the forward pass of the data based on the layers created above
        """
        out1 = self.softmax(self.l1(x))
        out2 = self.softmax(self.l2(out1))
        y_pred = self.softmax(self.l3(out2))
        return y_pred

In [62]:
# Creating the Dataset and Dataloader

class MyDataset(Dataset):
    
    def __init__(self):
        xy_df = pd.read_csv('iris.csv', delimiter=',')
        self.len = xy_df.shape[0]
        self.x_data = torch.from_numpy(xy_df.iloc[:,0:-1].to_numpy(dtype=np.float32))
        self.y_data = torch.from_numpy((pd.get_dummies(xy_df.iloc[:,-1])).values.argmax(1))
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len       

In [63]:
model = Model()
my_dataset = MyDataset()

In [64]:
train_loader = DataLoader(dataset = my_dataset, batch_size=16, shuffle=True, num_workers=0)

In [65]:
criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimus = torch.optim.SGD(model.parameters(), lr = 0.05)

In [66]:
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # getting the inputs and labels
        inputs, labels = data
#         print(f'labels: {labels.shape}')
        # Predictions from the model
        y_pred = model(inputs)
        
        # Get the loss
        loss = criterion(y_pred, labels)
        print(f'Epoch: {epoch}, Loss: {loss}')
        
        # optimus roll out
        optimus.zero_grad()
        loss.backward()
        optimus.step()
        

Epoch: 0, Loss: 1.0957831144332886
Epoch: 0, Loss: 1.0617023706436157
Epoch: 0, Loss: 1.1311962604522705
Epoch: 0, Loss: 1.1077425479888916
Epoch: 0, Loss: 1.1067123413085938
Epoch: 0, Loss: 1.1175349950790405
Epoch: 0, Loss: 1.0946459770202637
Epoch: 0, Loss: 1.0838195085525513
Epoch: 0, Loss: 1.1316670179367065
Epoch: 0, Loss: 1.103289246559143
Epoch: 1, Loss: 1.0978373289108276
Epoch: 1, Loss: 1.1299892663955688
Epoch: 1, Loss: 1.0927168130874634
Epoch: 1, Loss: 1.0956991910934448
Epoch: 1, Loss: 1.0979350805282593
Epoch: 1, Loss: 1.1084392070770264
Epoch: 1, Loss: 1.1073124408721924
Epoch: 1, Loss: 1.1177277565002441
Epoch: 1, Loss: 1.0823955535888672
Epoch: 1, Loss: 1.100244402885437


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
